In [17]:
import pandas as pd     
import numpy as np  

In [18]:
import preprocessor as p
import re
#from preprocessor.api import clean, tokenize, parse

In [19]:
from textblob import TextBlob
from textblob import Word

In [20]:
import nltk
from nltk.corpus import stopwords
from nltk.corpus import words

In [21]:
pd.options.display.max_colwidth=300

In [22]:
raw_data = pd.read_csv('data/raw/scomo.csv', usecols=['tweet_text'])
raw_data.head(20)

,tweet_text
0,b'@BelindaJones68 @Hillsong @ScottMorrisonMP @nswpolice Wonder if @nswpolice know about this and would they or Fuller take action?'
1,b'@Ctl_Alt_Del @cscviews @ScottMorrisonMP China doesn\xe2\x80\x99t need to raise a sweat it just buys everything'
2,b'@Ctl_Alt_Del @ScottMorrisonMP Wants to keep up the polls e.g. leaders during wartime popularity rises \xf0\x9f\xa4\xac'
3,b'@BelindaJones68 @ScottMorrisonMP @BrianCHouston @nswpolice \n@NSWHealth \nYou guys going to investigate?'
4,b'Now the rest of the world! Looking at you @ScottMorrisonMP @markrutte https://t.co/gGCTW7sa22'
5,"b""@isaacfloyd13 @Hillsong @ScottMorrisonMP @nswpolice He didn't speak in tongues at this event but it would be intere\xe2\x80\xa6 https://t.co/vUR7lNXMOp"""
6,b'@EDM17503975 @Sutts1102 @Milliganreports @poornhonest @DanielAndrewsMP @ScottMorrisonMP Why would the victims of th\xe2\x80\xa6 https://t.co/IJGT0t9wyc'
7,b'@ScottMorrisonMP @SecPompeo \xe2\x80\x9caUsTRaLiA and the US aRE gOod mAtEs\xe2\x80\x9d \n\nGet new line ya fucken nerd.'
8,"b'@KezBenn @SkyNewsAust @ScottMorrisonMP So you agree with the social restrictions on movements then? And, asking me\xe2\x80\xa6 https://t.co/fq5UuQvkMo'"
9,b'Cudos to Japan now Australia needs to do the same @ScottMorrisonMP @JoshFrydenberg you want to remain in Government\xe2\x80\xa6 https://t.co/op2TvJ9pYn'


# Tweet cleaning 
### Remove URL, mention, hashtags, reserved words, emoji, smiley and number

In [23]:
raw_data['tweet_text'] = np.array([ p.clean(tweet) for tweet in raw_data['tweet_text'] ])
raw_data.head(20)

,tweet_text
0,b' Wonder if know about this and would they or Fuller take action?'
1,b' China doesn\xe2\x80\x99t need to raise a sweat it just buys everything'
2,b' Wants to keep up the polls e.g. leaders during wartime popularity rises \xf0\x9f\xa4\xac'
3,b' \n \nYou guys going to investigate?'
4,b'Now the rest of the world! Looking at you '
5,"b"" He didn't speak in tongues at this event but it would be intere\xe2\x80\xa6 """
6,b' Why would the victims of th\xe2\x80\xa6 '
7,b' \xe2\x80\x9caUsTRaLiA and the US aRE gOod mAtEs\xe2\x80\x9d \n\nGet new line ya fucken nerd.'
8,"b' So you agree with the social restrictions on movements then? And, asking me\xe2\x80\xa6 '"
9,b'Cudos to Japan now Australia needs to do the same you want to remain in Government\xe2\x80\xa6 '


### Remove punctuations

In [24]:
raw_data['tweet_text'] = np.array([ re.sub(r'(@[A-Za-z0-9]+)|([^0)-9A-Za-z \t])|(\w+:\/\/\S+)|(\'|\"|\`|…)', "", tweet) for tweet in raw_data['tweet_text'] ])
raw_data.head(20)

,tweet_text
0,b Wonder if know about this and would they or Fuller take action
1,b China doesnxe2x80x99t need to raise a sweat it just buys everything
2,b Wants to keep up the polls e.g. leaders during wartime popularity rises xf0x9fxa4xac
3,b n nYou guys going to investigate
4,bNow the rest of the world Looking at you
5,b He didnt speak in tongues at this event but it would be interexe2x80xa6
6,b Why would the victims of thxe2x80xa6
7,b xe2x80x9caUsTRaLiA and the US aRE gOod mAtEsxe2x80x9d nnGet new line ya fucken nerd.
8,"b So you agree with the social restrictions on movements then And, asking mexe2x80xa6"
9,bCudos to Japan now Australia needs to do the same you want to remain in Governmentxe2x80xa6


### Remove first letter 'b'

In [25]:
raw_data['tweet_text'] = np.array([ tweet[1:] for tweet in raw_data['tweet_text'] ])
raw_data.head(20)

,tweet_text
0,Wonder if know about this and would they or Fuller take action
1,China doesnxe2x80x99t need to raise a sweat it just buys everything
2,Wants to keep up the polls e.g. leaders during wartime popularity rises xf0x9fxa4xac
3,n nYou guys going to investigate
4,Now the rest of the world Looking at you
5,He didnt speak in tongues at this event but it would be interexe2x80xa6
6,Why would the victims of thxe2x80xa6
7,xe2x80x9caUsTRaLiA and the US aRE gOod mAtEsxe2x80x9d nnGet new line ya fucken nerd.
8,"So you agree with the social restrictions on movements then And, asking mexe2x80xa6"
9,Cudos to Japan now Australia needs to do the same you want to remain in Governmentxe2x80xa6


### Lower case

In [26]:
raw_data['tweet_text'] = raw_data['tweet_text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
raw_data.head(20)

,tweet_text
0,wonder if know about this and would they or fuller take action
1,china doesnxe2x80x99t need to raise a sweat it just buys everything
2,wants to keep up the polls e.g. leaders during wartime popularity rises xf0x9fxa4xac
3,n nyou guys going to investigate
4,now the rest of the world looking at you
5,he didnt speak in tongues at this event but it would be interexe2x80xa6
6,why would the victims of thxe2x80xa6
7,xe2x80x9caustralia and the us are good matesxe2x80x9d nnget new line ya fucken nerd.
8,"so you agree with the social restrictions on movements then and, asking mexe2x80xa6"
9,cudos to japan now australia needs to do the same you want to remain in governmentxe2x80xa6


### Remove stop words

In [27]:
stop_words = stopwords.words('english')

In [28]:
raw_data['tweet_text'] = raw_data['tweet_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_words))
raw_data.head(20)

,tweet_text
0,wonder know would fuller take action
1,china doesnxe2x80x99t need raise sweat buys everything
2,wants keep polls e.g. leaders wartime popularity rises xf0x9fxa4xac
3,n nyou guys going investigate
4,rest world looking
5,didnt speak tongues event would interexe2x80xa6
6,would victims thxe2x80xa6
7,xe2x80x9caustralia us good matesxe2x80x9d nnget new line ya fucken nerd.
8,"agree social restrictions movements and, asking mexe2x80xa6"
9,cudos japan australia needs want remain governmentxe2x80xa6


### Rare words removal

In [29]:
freq = pd.Series(' '.join(raw_data['tweet_text']).split()).value_counts()[-100:]
freq = list(freq.index)
raw_data['tweet_text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
raw_data.head(20)

,tweet_text
0,wonder know would fuller take action
1,china doesnxe2x80x99t need raise sweat buys everything
2,wants keep polls e.g. leaders wartime popularity rises xf0x9fxa4xac
3,n nyou guys going investigate
4,rest world looking
5,didnt speak tongues event would interexe2x80xa6
6,would victims thxe2x80xa6
7,xe2x80x9caustralia us good matesxe2x80x9d nnget new line ya fucken nerd.
8,"agree social restrictions movements and, asking mexe2x80xa6"
9,cudos japan australia needs want remain governmentxe2x80xa6


In [30]:
freq = pd.Series(' '.join(raw_data['tweet_text']).split()).value_counts()[-100:]
freq = list(freq.index)
freq

['connection.',
 'nnwhilst',
 'reinforces',
 'sanger,',
 'firmly',
 'xe2x98x9dxf0x9fx8fxbband',
 'charitable',
 'lunchbox',
 'aerosolised',
 'fiercely',
 '.sounds',
 'mural.',
 'nbl',
 'trawl',
 'nn-us',
 'seekersxe2x80xa6',
 'ing-wen',
 'job.nnkeep',
 'xe2x80x9cduring',
 ',give',
 'xe2x80x9cwonderfulxe2x80x9d',
 'prefect',
 'nob.',
 'nest',
 'jeeezus',
 'participant',
 'cabixe2x80xa6',
 'deferring,',
 'three-hatted',
 'xf0x9fxa4xaexf0x9fxa4xa2making',
 'substantial.',
 'world)',
 'jar.',
 'angst',
 'hoisting',
 'sir...kindly',
 'die.xe2x80xa6',
 'past.xe2x80xa6',
 'xe2x80x98go',
 'bleed',
 'twist.',
 'whinger.',
 '.can',
 'eertsxe2x80xa6',
 'greenies',
 'ago,instead',
 'winterxe2x80xa6',
 'die...',
 'defencxe2x80xa6',
 'decates',
 'measure,',
 'muscatel',
 'depression-like',
 'xf0x9fx87xa6xf0x9fx87xbaxf0x9fx87xa6xf0x9fx87xbaxf0x9fx87xa6xf0x9fx87xbaxf0x9fx87xa6xf0x9fx87xbaxf0x9fx87xa6xf0x9fx87xbaxe2x80xa6',
 'tx',
 'struggling.xe2x80xa6',
 'gates/illumixe2x80xa6',
 'sacred...',
 'touch

### Lemmatization
#### we usually prefer using lemmatization over stemming

In [31]:
raw_data['tweet_text'] = raw_data['tweet_text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
raw_data.head(20)

,tweet_text
0,wonder know would fuller take action
1,china doesnxe2x80x99t need raise sweat buy everything
2,want keep poll e.g. leader wartime popularity rise xf0x9fxa4xac
3,n nyou guy going investigate
4,rest world looking
5,didnt speak tongue event would interexe2x80xa6
6,would victim thxe2x80xa6
7,xe2x80x9caustralia u good matesxe2x80x9d nnget new line ya fucken nerd.
8,"agree social restriction movement and, asking mexe2x80xa6"
9,cudos japan australia need want remain governmentxe2x80xa6


In [32]:
raw_data.head()

,tweet_text
0,wonder know would fuller take action
1,china doesnxe2x80x99t need raise sweat buy everything
2,want keep poll e.g. leader wartime popularity rise xf0x9fxa4xac
3,n nyou guy going investigate
4,rest world looking


### Import other data columns

In [33]:
data = pd.read_csv('data/raw/scomo.csv', usecols=['tweet_length','tweet_likes','tweet_retweets'])

In [34]:
processed_df = pd.concat([raw_data, data], axis=1)

In [35]:
processed_df.head()

,tweet_text,tweet_length,tweet_likes,tweet_retweets
0,wonder know would fuller take action,128,0,0
1,china doesnxe2x80x99t need raise sweat buy everything,99,0,0
2,want keep poll e.g. leader wartime popularity rise xf0x9fxa4xac,103,0,0
3,n nyou guy going investigate,102,0,0
4,rest world looking,93,0,0


# Write to csv

In [395]:
processed_df.to_csv('data/processed/processed_data.csv',index=False)

# Not Used

### Spelling correction

In [ ]:
raw_data['tweet_text'].apply(lambda x: str(TextBlob(x).correct()))
raw_data.head(20)

### Remove non English words
#### Not working yet

In [ ]:
raw_data1 = pd.read_csv('data/raw/scomo.csv', usecols=['tweet_text'], nrows=10)
raw_data1.head(20)

In [ ]:
raw_data1['tweet_text'] = raw_data1['tweet_text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
raw_data1.head(20)

In [ ]:
raw_data1['tweet_text'] = raw_data1['tweet_text'].apply(lambda x: " ".join(x for x in x.split() if x in words.words()))
raw_data1['tweet_text'].head(20)

### tokenize 

In [ ]:
for tweet in raw_data['tweet_text'].head(10):
    blob = TextBlob(tweet).words
    for w in blob:
        if w in words.words():
            print(w)
            

    
